In [19]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from datetime import datetime

## Constants

In [20]:
base_dir = Path('/Users/eandreas/projects/dev/covid-19/bag_data_download').resolve()

cases_dir = base_dir / 'downloads' / 'cases_data'
report_dir = base_dir / 'downloads' / 'report_data'
test_dir = base_dir / 'downloads' / 'test_data'
csv_dir = base_dir / 'csv'

## Methods

In [21]:
def get_latest(directory, prefix = '', suffix='', n = 1):
    time, latest = sorted((f.stat().st_mtime, f) for f in directory.glob(prefix + '*' + suffix))[-n]
    return latest

In [22]:
def load_data(nn = 1):
    # create DataFrame from Excel
    xlsx = get_latest(cases_dir, prefix='2020', suffix='.xlsx', n = nn)
    print(xlsx)
    df = pd.read_excel(xlsx)
    
    renames = {
        'replikation_dt': 'date',
        'fall_dt' : 'case_date',
        'ktn': 'canton',
        'akl': 'age_class',
        'fallklasse_3': 'conf',
        'pttod_1': 'deceased',
        'pttoddat': 'deceased_date'
    }
    
    # rename columns
    df.rename(columns = renames,inplace=True)
    
    # split datetime column into a date and a time column
    #time_list = pd.to_datetime(df['date'], dayfirst=True).dt.time
    #df.insert(loc=1, column='time', value=time_list)                 
    #df['date'] = pd.to_datetime(df['date'], dayfirst=True).dt.date
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df['case_date'] = pd.to_datetime(df['case_date'], dayfirst=True).dt.date
    df['case_date'] = pd.to_datetime(df['case_date'], dayfirst=True)
    df['deceased_date'] = pd.to_datetime(df['deceased_date'], dayfirst=True).dt.date
    df['deceased_date'] = pd.to_datetime(df['deceased_date'], dayfirst=True)

    # clean up
    df.loc[:,'sex'] = np.where(df['sex'] == 1, 'm', np.where(df['sex'] == 2, 'f', 'n/a'))
    df.drop('Geschlecht', axis=1, inplace=True)
    df.drop('Sexe', axis=1, inplace=True)

    # insert column for country
    df.insert(loc=3, column='country', value=np.where(df['canton'] == 'FL', 'FL', 'CH'))

    return df

## Laden der BAG-Daten

In [23]:
df_latest = load_data(1)
df_second_latest = load_data(2)

/Users/eandreas/projects/dev/covid-19/bag_data_download/downloads/cases_data/2020-10-28_12-00_Dashboards_1&2_COVID19_swiss_data_pv.xlsx
/Users/eandreas/projects/dev/covid-19/bag_data_download/downloads/cases_data/2020-10-27_12-00_Dashboards_1&2_COVID19_swiss_data_pv.xlsx


## Neue und bisherige Fälle / Tag - ganze Schweiz

In [24]:
index = pd.MultiIndex.from_frame(df_latest[['case_date']])
df_ch = df_latest.copy()
df_ch.set_index(index, inplace=True)
df_ch = df_ch.drop(['date', 'case_date', 'age_class', 'sex', 'deceased_date', 'deceased'], axis=1).reset_index()
df_ch = df_ch.groupby('case_date').sum().reset_index()

In [25]:
index = pd.MultiIndex.from_frame(df_second_latest[['case_date']])
df_ch_y = df_second_latest.copy()
df_ch_y.set_index(index, inplace=True)
df_ch_y = df_ch_y.drop(['date', 'case_date', 'age_class', 'sex', 'deceased_date', 'deceased'], axis=1).reset_index()
df_ch_y = df_ch_y.groupby('case_date').sum().reset_index()
df_ch_y.rename(columns = {'conf': 'conf_yesterday'}, inplace=True)

In [26]:
df_ch = df_ch.merge(df_ch_y, how='left', on='case_date')
df_ch.fillna(value = 0, inplace=True)
df_ch['new_conf'] = (df_ch.conf - df_ch.conf_yesterday).astype(int)
df_ch.drop(columns=['conf_yesterday'], axis = 1, inplace = True)

In [27]:
fig = go.Figure(data=[
    go.Bar(x=df_ch.case_date, y=df_ch.conf-df_ch.new_conf, marker_color='indianred', name = 'bisher'),
    go.Bar(x=df_ch.case_date, y=df_ch.new_conf, marker_color='lightsalmon', name = 'neu')
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

## Write CSV

In [28]:
df_latest.head()

,date,case_date,canton,country,age_class,sex,conf,deceased_date,deceased
0,2020-10-28 07:36:35,2020-02-24,TI,CH,70 - 79,m,1,NaT,0
1,2020-10-28 07:36:35,2020-02-25,AG,CH,20 - 29,m,1,NaT,0
2,2020-10-28 07:36:35,2020-02-26,BL,CH,20 - 29,m,1,NaT,0
3,2020-10-28 07:36:35,2020-02-26,BS,CH,20 - 29,f,1,NaT,0
4,2020-10-28 07:36:35,2020-02-26,GE,CH,20 - 29,m,1,NaT,0


In [29]:
df_latest.to_csv(csv_dir / 'cases_all.csv', index = False)

In [12]:
url = 'https://raw.githubusercontent.com/eandreas/bag_data_download/master/csv/cases_all.csv'

In [13]:
df_tmp = pd.read_csv(url, parse_dates=[['date', 'time'], 'case_date', 'deceased_date'])
df_tmp.head()

,date_time,case_date,country,canton,age_class,sex,conf,deceased_date,deceased
0,2020-10-23 07:52:18,2020-02-24,CH,TI,70 - 79,m,1,NaT,0
1,2020-10-23 07:52:18,2020-02-25,CH,AG,20 - 29,m,1,NaT,0
2,2020-10-23 07:52:18,2020-02-26,CH,BL,20 - 29,m,1,NaT,0
3,2020-10-23 07:52:18,2020-02-26,CH,BS,20 - 29,f,1,NaT,0
4,2020-10-23 07:52:18,2020-02-26,CH,GE,20 - 29,m,1,NaT,0


## Playgrounds

In [30]:
df_ch = df_latest.copy()

In [31]:
df_c = pd.DataFrame()
df_c['date'] = df_ch['case_date']
df_c['country'] = df_ch['country']
df_c['canton'] = df_ch['canton']
df_c['age_class'] = df_ch['age_class']
df_c['sex'] = df_ch['sex']
df_c['conf'] = df_ch['conf']
df_c = df_c[df_c.date.notnull()]

In [32]:
df_c.head()

,date,country,canton,age_class,sex,conf
0,2020-02-24,CH,TI,70 - 79,m,1
1,2020-02-25,CH,AG,20 - 29,m,1
2,2020-02-26,CH,BL,20 - 29,m,1
3,2020-02-26,CH,BS,20 - 29,f,1
4,2020-02-26,CH,GE,20 - 29,m,1


In [33]:
df_d = pd.DataFrame()
df_d['date'] = df_ch['deceased_date']
df_d['country'] = df_ch['country']
df_d['canton'] = df_ch['canton']
df_d['age_class'] = df_ch['age_class']
df_d['sex'] = df_ch['sex']
df_d['deceased'] = df_ch['deceased']
df_d = df_d[df_d.date.notnull()]

In [34]:
df_d.head()

,date,country,canton,age_class,sex,deceased
200880,2020-05-26,CH,AG,0 - 9,m,1
200888,2020-03-28,CH,AG,30 - 39,f,1
200892,2020-05-01,CH,AG,40 - 49,f,1
200895,2020-08-23,CH,AG,50 - 59,m,1
200896,2020-09-11,CH,AG,50 - 59,m,1


In [35]:
df_c_d = pd.merge(df_c, df_d, how='outer', on=['date', 'country', 'canton', 'age_class', 'sex'])
df_c_d.fillna(0, inplace=True)
df_c_d.deceased = df_c_d.deceased.astype(int)

In [36]:
df_c_d.head()

,date,country,canton,age_class,sex,conf,deceased
0,2020-02-24,CH,TI,70 - 79,m,1,0
1,2020-02-25,CH,AG,20 - 29,m,1,0
2,2020-02-26,CH,BL,20 - 29,m,1,0
3,2020-02-26,CH,BS,20 - 29,f,1,0
4,2020-02-26,CH,GE,20 - 29,m,1,0


In [37]:
df_c.conf.sum(), df_d.deceased.sum(), df_c_d.conf.sum(), df_c_d.deceased.sum()

(135658, 1954, 135658, 1954)

In [38]:
df_c_d.to_csv(csv_dir / 'cases_all_indexed.csv', index = False)

In [39]:
# all cantons including FL
CANTONS = {
    1: "AG",
    2: "AI",
    3: "AR",
    4: "BE",
    5: "BL",
    6: "BS",
    7: "FL",
    8: "FR",
    9: "GE",
    10: "GL",
    11: "GR",
    12: "JU",
    13: "LU",
    14: "NE",
    15: "NW",
    16: "OW",
    17: "SG",
    18: "SH",
    19: "SO",
    20: "SZ",
    21: "TG",
    22: "TI",
    23: "UR",
    24: "VD",
    25: "VS",
    26: "ZG",
    27: "ZH"
}

AGE_CLASSES = {
    '0-9': '0 - 9',
    '10-19': '10 - 19',
    '20-29': '20 - 29',
    '30-39': '30 - 39',
    '40-49': '40 - 49',
    '50-59': '50 - 59',
    '60-69': '60 - 69',
    '70-79': '70 - 79',
    '80+': '80+',
    'ukn': 'Unbekannt'
}

SEX = {
    'f': 'f',
    'm': 'm',
    'ukn': 'n/a'
}

VALUES = ['conf', 'dead']

In [40]:
#idx = list()
df_final = pd.DataFrame(columns=['date'])

for c in CANTONS.values():
    for s_k, s_v in SEX.items():
        for a_k, a_v in AGE_CLASSES.items():
            df_tmp = df_c_d[
                (df_c_d.canton == c) & 
                (df_c_d.sex == s_v) & 
                (df_c_d.age_class == a_v)].groupby('date').sum()
            df_final = pd.merge(df_final, df_tmp, how='outer', on=['date'])
            bn = f'{c}_{s_k}_{a_k}'
            print(bn)
            df_final.rename(columns = {
                'conf': f'{bn}_conf','deceased': f'{bn}_dead'},inplace=True)

AG_f_0-9
AG_f_10-19
AG_f_20-29
AG_f_30-39
AG_f_40-49
AG_f_50-59
AG_f_60-69
AG_f_70-79
AG_f_80+
AG_f_ukn
AG_m_0-9
AG_m_10-19
AG_m_20-29
AG_m_30-39
AG_m_40-49
AG_m_50-59
AG_m_60-69
AG_m_70-79
AG_m_80+
AG_m_ukn
AG_ukn_0-9
AG_ukn_10-19
AG_ukn_20-29
AG_ukn_30-39
AG_ukn_40-49
AG_ukn_50-59
AG_ukn_60-69
AG_ukn_70-79
AG_ukn_80+
AG_ukn_ukn
AI_f_0-9
AI_f_10-19
AI_f_20-29
AI_f_30-39
AI_f_40-49
AI_f_50-59
AI_f_60-69
AI_f_70-79
AI_f_80+
AI_f_ukn
AI_m_0-9
AI_m_10-19
AI_m_20-29
AI_m_30-39
AI_m_40-49
AI_m_50-59
AI_m_60-69
AI_m_70-79
AI_m_80+
AI_m_ukn
AI_ukn_0-9
AI_ukn_10-19
AI_ukn_20-29
AI_ukn_30-39
AI_ukn_40-49
AI_ukn_50-59
AI_ukn_60-69
AI_ukn_70-79
AI_ukn_80+
AI_ukn_ukn
AR_f_0-9
AR_f_10-19
AR_f_20-29
AR_f_30-39
AR_f_40-49
AR_f_50-59
AR_f_60-69
AR_f_70-79
AR_f_80+
AR_f_ukn
AR_m_0-9
AR_m_10-19
AR_m_20-29
AR_m_30-39
AR_m_40-49
AR_m_50-59
AR_m_60-69
AR_m_70-79
AR_m_80+
AR_m_ukn
AR_ukn_0-9
AR_ukn_10-19
AR_ukn_20-29
AR_ukn_30-39
AR_ukn_40-49
AR_ukn_50-59
AR_ukn_60-69
AR_ukn_70-79
AR_ukn_80+
AR_ukn_ukn
BE_f

VS_ukn_20-29
VS_ukn_30-39
VS_ukn_40-49
VS_ukn_50-59
VS_ukn_60-69
VS_ukn_70-79
VS_ukn_80+
VS_ukn_ukn
ZG_f_0-9
ZG_f_10-19
ZG_f_20-29
ZG_f_30-39
ZG_f_40-49
ZG_f_50-59
ZG_f_60-69
ZG_f_70-79
ZG_f_80+
ZG_f_ukn
ZG_m_0-9
ZG_m_10-19
ZG_m_20-29
ZG_m_30-39
ZG_m_40-49
ZG_m_50-59
ZG_m_60-69
ZG_m_70-79
ZG_m_80+
ZG_m_ukn
ZG_ukn_0-9
ZG_ukn_10-19
ZG_ukn_20-29
ZG_ukn_30-39
ZG_ukn_40-49
ZG_ukn_50-59
ZG_ukn_60-69
ZG_ukn_70-79
ZG_ukn_80+
ZG_ukn_ukn
ZH_f_0-9
ZH_f_10-19
ZH_f_20-29
ZH_f_30-39
ZH_f_40-49
ZH_f_50-59
ZH_f_60-69
ZH_f_70-79
ZH_f_80+
ZH_f_ukn
ZH_m_0-9
ZH_m_10-19
ZH_m_20-29
ZH_m_30-39
ZH_m_40-49
ZH_m_50-59
ZH_m_60-69
ZH_m_70-79
ZH_m_80+
ZH_m_ukn
ZH_ukn_0-9
ZH_ukn_10-19
ZH_ukn_20-29
ZH_ukn_30-39
ZH_ukn_40-49
ZH_ukn_50-59
ZH_ukn_60-69
ZH_ukn_70-79
ZH_ukn_80+
ZH_ukn_ukn


In [41]:
df_final.tail()

,date,AG_f_0-9_conf,AG_f_0-9_dead,AG_f_10-19_conf,AG_f_10-19_dead,AG_f_20-29_conf,AG_f_20-29_dead,AG_f_30-39_conf,AG_f_30-39_dead,AG_f_40-49_conf,...,ZH_ukn_50-59_conf,ZH_ukn_50-59_dead,ZH_ukn_60-69_conf,ZH_ukn_60-69_dead,ZH_ukn_70-79_conf,ZH_ukn_70-79_dead,ZH_ukn_80+_conf,ZH_ukn_80+_dead,ZH_ukn_ukn_conf,ZH_ukn_ukn_dead
243,2020-10-24,0,0,13,0,23,0,16,0,19,...,0,0,0,0,0,0,0,0,0,0
244,2020-10-25,0,0,3,0,13,0,12,0,9,...,0,0,0,0,0,0,0,0,0,0
245,2020-10-26,3,0,13,0,35,0,29,0,18,...,0,0,0,0,0,0,0,0,0,0
246,2020-10-27,0,0,7,0,12,0,11,0,12,...,0,0,0,0,0,0,0,0,0,0
247,2020-10-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df_final.to_csv(csv_dir / 'cases_all_new.csv', index = False)

In [17]:
df_d.deceased.sum()

1954

In [18]:
df_latest.deceased.sum()

1954

In [44]:
df_final.replace(0, np.nan, inplace=True)

In [45]:
df_final.to_csv(csv_dir / 'cases_all_new_nan.csv', index = False)